In [3]:
import pandas as pd
import nltk
import spacy
import json
import os
import re
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
import locationtagger
import geonamescache

# essential entity models downloads
nltk.downloader.download('maxent_ne_chunker')
nltk.downloader.download('words')
nltk.downloader.download('treebank')
nltk.downloader.download('maxent_treebank_pos_tagger')
nltk.downloader.download('punkt')
nltk.download('averaged_perceptron_tagger')


[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\zoeje\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\zoeje\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package treebank to
[nltk_data]     C:\Users\zoeje\AppData\Roaming\nltk_data...
[nltk_data]   Package treebank is already up-to-date!
[nltk_data] Downloading package maxent_treebank_pos_tagger to
[nltk_data]     C:\Users\zoeje\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_treebank_pos_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\zoeje\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\zoeje\AppData\Roaming\nltk_data...
[nltk_data]   Package avera

True

In [4]:
df = pd.DataFrame([], columns=['book_id', 'hierarchy', 'language', 'nr_pages', 'word_count'])

path = 'transcripts/no_doubles/'

for filename in os.listdir(path):
    book = json.load(open(path + filename))
    data = {'book_id': filename.replace('.json', ''),
            'hierarchy': book['hierarchy'],
            'language': book['language'],
            'nr_pages': len(book['pages'])}
    
    
    # get the word count per book:
    data['word_count'] = 0
    for page in book['pages']:
        # if there are multiple transcriptions, we don't know which one to use
        if len(page) > 1:
            print('MULTIPLE TRANSCRIPTIONS FOUND: ' + filename)
            break
        else:
            p = page[0]
            for sentence in p:
                # remove all non-alphabetic characters
                data['word_count'] += len(re.sub('[^a-zA-Z ]+', '', sentence).split())

    data=pd.DataFrame(data, index=[0])
    df = pd.concat([df, data], axis=0, ignore_index=True)

In [5]:
meta_df = pd.read_csv('travel_df.csv', index_col =False,encoding='utf-8')

In [6]:
meta_df

,Unnamed: 0,book_id,hierarchy,language,nr_pages,word_count,title,author,year,opac,...,transkribus,id,subtitle,statement,edition,volume,place,publisher,contents,illustrations
0,0,824777,Rome/Rome_after_1800/Praktisches Reise-Handbuc...,deu,654,1592068,Praktisches Reise-Handbuch nach und durch Italien,"Lewald, August",1840.0,https://aleph.mpg.de/F/?func=find-b&local_base...,...,https://transkribus.humanitiesconnect.pub/#/do...,BV001611431,mit Berücksichtigung aller dem Reisenden nothw...,von August Lewald,NaN,NaN,Stuttgart,Hoffmann,"XIV, 608 S.",Kt.
1,1,825300,Rome/Rome_after_1800/Anatomia per uso dei pitt...,ita_old,170,144696,Anatomia per uso dei pittori e scultori,"Medico, Giuseppe del",1811.0,https://aleph.mpg.de/F/?func=find-b&local_base...,...,https://transkribus.humanitiesconnect.pub/#/do...,BV001535558,NaN,NaN,NaN,NaN,Roma,NaN,NaN,NaN
2,2,825301,Rome/Rome_after_1800/Nuovissima guida dei viag...,ita_old,578,979809,Nuovissima guida dei viaggiatori in Italia,"Artaria, Epimaco",1834.0,https://aleph.mpg.de/F/?func=find-b&local_base...,...,https://transkribus.humanitiesconnect.pub/#/do...,BV037781541,arricchita di carte geografiche generali e pos...,(ed.: Epimaco e Pasquale Artaria),"3. ed., notabilmente accresciuta e corretta",NaN,Milano,Artaria [u.a.],"XIV, 480 S., [12] Bl.","Ill., Kt."
3,3,856536,Rome/Rome_after_1800/Les Delices De L'Italie |...,fra,395,307163,Les Delices De L'Italie,"Rogissart, Alexandre de",1709.0,https://aleph.mpg.de/F/?func=find-b&local_base...,...,https://transkribus.humanitiesconnect.pub/#/do...,BV010911277,Qui contiennent une Description exacte du Pays...,"Par les S.rs De Rogissart, & H.***","Dernière Edition, revue, augmentée de nouveau,...",2,Leide,Vander Aa,"[1] Bl., 147 S., [1] Bl., [9] gef. Bl. : Kupfe...",NaN
4,4,856537,Rome/Rome_after_1800/Les Delices De L'Italie |...,fra,424,327528,Les Delices De L'Italie,"Rogissart, Alexandre de",1709.0,https://aleph.mpg.de/F/?func=find-b&local_base...,...,https://transkribus.humanitiesconnect.pub/#/do...,BV010911311,Qui contiennent une Description exacte du Pays...,"Par les S.rs De Rogissart, & H.***","Dernière Edition, revue, augmentée de nouveau,...",6,Leide,Vander Aa,"[1] Bl., 209 S., [65] Bl., [9] Bl., [36] gef. ...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,73,867652,Rome/Rome_before_1800/Letters from a young pai...,eng,438,440469,Letters from a young painter abroad to his fri...,"Russell, John",1750.0,https://aleph.mpg.de/F/?func=find-b&local_base...,...,https://transkribus.humanitiesconnect.pub/#/do...,BV038821999,adorned with copper plates,[Verf.: John Russell],2. ed.,2,London,Russell,NaN,NaN
74,74,867653,Rome/Rome_before_1800/Letters from Italy | des...,eng,328,300304,Letters from Italy,"Sharp, Samuel",1767.0,https://aleph.mpg.de/F/?func=find-b&local_base...,...,https://transkribus.humanitiesconnect.pub/#/do...,BV001648510,describing the customs and manners of that cou...,NaN,3. ed.,NaN,London,Henry and Cave,"IV, 315 S.",NaN
75,75,867654,Rome/Rome_before_1800/Travels through Italy/BV...,eng,516,772738,Travels through Italy,"Northall, John",1766.0,https://aleph.mpg.de/F/?func=find-b&local_base...,...,https://transkribus.humanitiesconnect.pub/#/do...,BV001463981,NaN,NaN,NaN,NaN,London,NaN,NaN,NaN
76,76,867655,Rome/Rome_before_1800/An extract from the obse...,eng,206,141865,An extract from the observations made in a tou...,"La Condamine, Charles-Marie de",1768.0,https://aleph.mpg.de/F/?func=find-b&local_base...,...,https://transkribus.humanitiesconnect.pub/#/do...,BV040637527,"by the Chevalier de la Condamine, ... Translat...",NaN,NaN,NaN,London,Newbery and Carnan,"II, 187 S.",NaN


In [7]:
exp_cols = ['book_id', 'language', 'title', 'year', 'author', 'place']
meta_df[exp_cols].to_csv('../website/src/data/books.csv', index=False, encoding='utf-8')


# Extracting cities


#### Some plotting 

In [8]:
def plot_top_30(texts):

    for id in texts: 

        cities = []

        for i in texts[id]['cities']: 
            cities.extend(texts[id]['cities'][i])
            
        counted_values = Counter(cities)

        # sort the counted values in descending order
        counted_values = dict(sorted(counted_values.items(), key=lambda item: item[1], reverse=True)[:30])  

        fig = plt.figure(figsize=(50, 15))
        
        plt.bar(counted_values.keys(), counted_values.values())
        # set x-label and y-label
        plt.xlabel('Values')
        plt.ylabel('Count')
        plt.title('Top 30 extracted place names in ' + str(id), fontsize=40)

        plt.xticks(rotation=45, ha='right', fontsize=40)
        plt.yticks(fontsize=40)
        plt.show()

In [9]:
def plot_rome_florence(texts): 
    
    
    for id in texts: 

        rome_by_page = []
        florence_by_page = []

        for page, cities in texts[id]['cities'].items(): 
                rome_count = cities.count('Rome')
                florence_count = cities.count('Florence')

                rome_by_page.append((page, rome_count))
                florence_by_page.append((page, florence_count))
            
        fig = plt.figure(figsize=(8, 6))
        #plt.stackplot([x[0] for x in rome_by_page], [x[1] for x in rome_by_page], [x[1] for x in florence_by_page], labels=['Rome', 'Florence'])

        plt.plot([x[0] for x in rome_by_page], [x[1] for x in rome_by_page], 'r-', label='Rome count')
        plt.plot([x[0] for x in florence_by_page], [x[1] for x in florence_by_page], 'b-', label='Florence count')

        # set x-label and y-label
        plt.xlabel('Page number')
        plt.ylabel('Number of mentions')

        # add legend
        plt.legend(loc='upper left')

        plt.show()

### Extracting all the english texts

In [10]:

path = 'transcripts/no_doubles/'

#dictionary containing the bookid and a list of text 

english_df = meta_df[meta_df.language == 'eng']
texts = {}


for bookid in english_df.book_id:
    filename = path + str(bookid) + '.json'

    book = json.load(open(filename))

    text = []

    for page in book['pages']:
        # if there are multiple transcriptions, we don't know which one to use
        if len(page) > 1:
            print('MULTIPLE TRANSCRIPTIONS FOUND: ' + filename)
            break
        else:
            if(page != ''):
                text += page

    texts[bookid] = {'text': text}

# SPACY

In [11]:
nlp = spacy.load("en_core_web_sm")

nlp.get_pipe("ner")

for id in texts: 
    
    texts[id]['cities'] = {}

    for i in range(len(texts[id]['text'])):

        texts[id]['cities'][i] = []
        text = texts[id]['text'][i]
        if(text != ''):
            
            doc = nlp(text)
            for ent in doc.ents:
                #Geo Political entity
                if ent.label_ == "GPE":
                    #print(ent.text)
                    texts[id]['cities'][i].append(ent.text)



In [12]:
# geonameCache for verification
gc = geonamescache.GeonamesCache()

# gets nested dictionary for countries
countries = gc.get_countries()

# gets nested dictionary for cities
cities = gc.get_cities()

In [13]:
def gen_dict_extract(var, key):
    if isinstance(var, dict):
        for k, v in var.items():
            if k == key:
                yield v
            if isinstance(v, (dict, list)):
                yield from gen_dict_extract(v, key)
    elif isinstance(var, list):
        for d in var:
            yield from gen_dict_extract(d, key)

In [14]:
cities_geo = [*gen_dict_extract(cities, 'name')]

In [15]:
cities_geo

['les Escaldes',
 'Andorra la Vella',
 'Umm Al Quwain City',
 'Ras Al Khaimah City',
 'Zayed City',
 'Khawr Fakkān',
 'Dubai',
 'Dibba Al-Fujairah',
 'Dibba Al-Hisn',
 'Sharjah',
 'Ar Ruways',
 'Al Fujairah City',
 'Al Ain City',
 'Ajman City',
 'Adh Dhayd',
 'Abu Dhabi',
 'Khalifah A City',
 'Bani Yas City',
 'Musaffah',
 'Al Shamkhah City',
 'Reef Al Fujairah City',
 'Zaranj',
 'Bāzār-e Yakāwlang',
 'Taloqan',
 'Shīnḏanḏ',
 'Shibirghān',
 'Shahrak',
 'Sar-e Pul',
 'Sang-e Chārak',
 'Aībak',
 'Rustāq',
 'Qarqīn',
 'Qarāwul',
 'Pul-e Khumrī',
 'Paghmān',
 'Nahrīn',
 'Maymana',
 'Mehtar Lām',
 'Mazār-e Sharīf',
 'Lashkar Gāh',
 'Kushk',
 'Kunduz',
 'Khōst',
 'Khulm',
 'Khāsh',
 'Khanabad',
 'Karukh',
 'Kandahār',
 'Kabul',
 'Jalālābād',
 'Jabal os Saraj',
 'Herāt',
 'Ghormach',
 'Ghazni',
 'Gereshk',
 'Gardez',
 'Fayzabad',
 'Farah',
 'Kafir Qala',
 'Charikar',
 'Baraki Barak',
 'Bāmyān',
 'Balkh',
 'Baghlān',
 'Ārt Khwājah',
 'Āsmār',
 'Asadābād',
 'Andkhōy',
 'Bāzārak',
 'Markaz-e Wol

In [16]:
locations = {}
for id in texts:
    locations[id] = texts[id]['cities']

In [17]:
with open('../website/src/data/locations_per_page.json', 'w') as f:
    json.dump(locations, f)

In [18]:
locations.items()

dict_items([(856797, {0: [], 1: [], 2: [], 3: [], 4: [], 5: ['LONDON'], 6: [], 7: [], 8: ['Gymnasium', 'Rome', 'Buffoons', 'Patronage', 'Buffoons', 'Rome', 'Berni', 'Beroaldo', 'Bibbiena', 'Acciajuoli', 'Rome'], 9: ['Rome', 'Sienna', 'Rome', 'Labours', 'Sculpture', 'Sculpture', 'Frescoes', "St. Peter's", 'Germany', 'Italy', 'Durer', 'Lucas van Leyden', 'Rome'], 10: ['Rome', 'Naples', 'Tansillo', 'Rome', 'Attachment', 'Boccaccio', 'Writings'], 11: ['Theobaldo'], 12: ['Ducas', 'Gymnasium', 'Rome.—Passes', 'Buffoons', 'Rome', 'Rome', 'Berni', 'Beroaldo', 'Bibbiena', 'Acciajuoli'], 13: [], 14: ['Italy', 'Greece'], 15: ['Italy', 'Paris', 'France', 'Venice', 'Rome'], 16: ['Venice', 'Rome', 'Rome', 'Lascaris', 'Greece', 'Candia'], 17: ['Venice', 'Venice', 'Rome', 'Sienna', 'Rome'], 18: ['Demetrius Ducas', 'Candia', 'Rome', 'Paris', 'Greece'], 19: ['Gymnasium', 'Rome'], 20: ['Venice'], 21: ['Rome', 'Paris', 'Rome', 'Rome', 'Rome', 'Greece'], 22: [], 23: ['Padua', 'Venice'], 24: ['Rome', 'Bembo

In [19]:
book_city = {}

for book_id, pages in locations.items():
    city_frequency = {}
    for page, cities in pages.items():
        for city in cities:
            # only getting clean cities for our lovely data viz
            if city in cities_geo:
                if city not in city_frequency:
                    city_frequency[city] = 1
                else:
                    city_frequency[city] += 1

    book_city[book_id] = city_frequency

In [20]:
book_city

{856797: {'Rome': 164,
  'Naples': 27,
  'Paris': 12,
  'Venice': 31,
  'Padua': 5,
  'Milan': 20,
  'Athens': 5,
  'Florence': 20,
  'Verona': 8,
  'Messina': 1,
  'Imperia': 8,
  'Nice': 2,
  'Orlando': 1,
  'Spring': 1,
  'Toledo': 1,
  'Belluno': 1,
  'Blois': 1,
  'Orleans': 1,
  'Avignon': 4,
  'Porta': 2,
  'Jerusalem': 2,
  'St. Thomas': 1,
  'Hercules': 1,
  'Carrara': 2,
  'Pietrasanta': 1,
  'Ferrara': 3,
  'Viterbo': 1,
  'Franconia': 1,
  'Cadiz': 1,
  'Terracina': 1,
  'Bologna': 1,
  'Lucca': 2,
  'Palermo': 2,
  'Frederick': 1,
  'Spoleto': 1,
  'Rota': 1,
  'Vittoria': 7,
  'Pavia': 4,
  'Brescia': 1,
  'Terni': 2,
  'Perugia': 1,
  'Rimini': 1,
  'Sparta': 1,
  'Ancona': 1,
  'San Lorenzo': 1,
  'Santa Maria': 1,
  'Amsterdam': 1,
  'Edinburgh': 1},
 856798: {'Porta': 3,
  'Rome': 97,
  'Florence': 11,
  'Romulus': 1,
  'Lucca': 1,
  'Naples': 5,
  'Milan': 38,
  'Venice': 114,
  'Paris': 7,
  'Padua': 15,
  'Genoa': 1,
  'Palestine': 1,
  'Algiers': 1,
  'Bologna': 8

In [21]:
with open('../website/src/data/locations_per_work.json', 'w') as f:
    json.dump(book_city, f)

In [ ]:
for book_id, pages in locations.items():
    cities_per_page={}
    for 
    

    




#with open('../data/transcripts/cities_per_page/' + str(book_id) + '.json', 'w') as f:
#    json.dump(cities_per_page, f)